## Ultralytics YOLOv8 

<a target="_blank" href="https://colab.research.google.com/github/dyi-el/optimized-traffic-light-computer-vision/blob/main/YOLOv8.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

For training using Google Colab

### Setup
Install and check dependencies

In [ ]:
%pip install fiftyone
%pip install ultralytics
import ultralytics
ultralytics.checks()

Import Google Drive and unzip the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip /content/drive/MyDrive/yolo.zip -d /content/yolo

COCO Dataset is filtered through FiftyOne https://docs.voxel51.com/user_guide/dataset_zoo/datasets.html#coco-2017

### Train

In [ ]:
logger = 'Comet' #@param ['Comet', 'TensorBoard']

if logger == 'Comet':
  %pip install -q comet_ml
  import comet_ml; comet_ml.init()
elif logger == 'TensorBoard':
  %load_ext tensorboard
  %tensorboard --logdir .

Move the dataset.yaml to the working directory.

In [ ]:

# Training mode
!yolo train model=yolov8n.pt data=/content/drive/MyDrive/dataset.yaml epochs=100 imgsz=640

### Validation

In [ ]:
# Validation mode
!yolo val model=yolov8n.pt data=dataset.yaml

### Export

In [ ]:
!yolo export model=yolov8n.pt format=torchscript